In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from PIL import Image

In [3]:
num_classes = 2
learning_rate = 0.001
dropout_rate = 0.5
batch_size = 32
num_epochs = 25

In [4]:
class GoogLeNetIrisTumor(nn.Module):
    def __init__(self, num_classes, dropout_rate):
        super(GoogLeNetIrisTumor, self).__init__()
        # Load GoogLeNet backbone
        self.googlenet = models.googlenet(pretrained=True)

        # Replace the fully connected layer to match the number of classes
        self.googlenet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.googlenet.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.googlenet(x)

In [5]:
model = GoogLeNetIrisTumor(num_classes=num_classes, dropout_rate=dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # GoogLeNet typically uses 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
class CustomDataset(Dataset):
    def __init__(self, folder_path, label, transform=None):
        self.folder_path = folder_path
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

In [7]:
tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/YES"
no_tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/NO"

In [8]:
tumor_dataset = CustomDataset(tumor_path, label=1, transform=transform)
no_tumor_dataset = CustomDataset(no_tumor_path, label=0, transform=transform)
combined_dataset = ConcatDataset([tumor_dataset, no_tumor_dataset])
train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_data, val_data = random_split(combined_dataset, [train_size, val_size])

In [9]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100. * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cpu'), labels.to('cpu')

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_epoch_acc = 100. * val_correct / val_total
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')



Epoch 1/10, Loss: 0.3987, Accuracy: 80.85%
Validation Loss: 0.4083, Validation Accuracy: 78.72%
Epoch 2/10, Loss: 0.0360, Accuracy: 99.47%
Validation Loss: 0.4044, Validation Accuracy: 89.36%
Epoch 3/10, Loss: 0.0397, Accuracy: 98.40%
Validation Loss: 0.1127, Validation Accuracy: 95.74%
Epoch 4/10, Loss: 0.0774, Accuracy: 97.34%
Validation Loss: 0.5837, Validation Accuracy: 93.62%
Epoch 5/10, Loss: 0.0104, Accuracy: 100.00%
Validation Loss: 0.6600, Validation Accuracy: 91.49%
Epoch 6/10, Loss: 0.0050, Accuracy: 100.00%
Validation Loss: 0.4520, Validation Accuracy: 91.49%
Epoch 7/10, Loss: 0.0016, Accuracy: 100.00%
Validation Loss: 0.2876, Validation Accuracy: 93.62%
Epoch 8/10, Loss: 0.0010, Accuracy: 100.00%
Validation Loss: 0.2170, Validation Accuracy: 97.87%
Epoch 9/10, Loss: 0.0013, Accuracy: 100.00%
Validation Loss: 0.1871, Validation Accuracy: 97.87%
Epoch 10/10, Loss: 0.0013, Accuracy: 100.00%
Validation Loss: 0.1789, Validation Accuracy: 95.74%


In [10]:
torch.save(model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Googlenet.pth')
